#Discrete wavelet transform

In [ ]:
import pywt
import numpy as np
import matplotlib.pyplot as plt
import random
import cv2

def channelTransform(ch1, ch2, shape):
    cooef1 = pywt.dwt2(ch1, 'sym8', mode='symmetric')
    cooef2 = pywt.dwt2(ch2, 'sym8', mode='symmetric')
    cA1, (cH1, cV1, cD1) = cooef1
    cA2, (cH2, cV2, cD2) = cooef2

    cA = (cA1 + cA2) / 2
    cH = (cH1 + cH2) / 2
    cV = (cV1 + cV2) / 2
    cD = (cD1 + cD2) / 2
    fincoC = cA, (cH, cV, cD)
    outImageC = pywt.idwt2(fincoC, 'sym8', mode='symmetric')
    outImageC = cv2.resize(outImageC, (shape[1], shape[0]))
    return outImageC

def fusion(img1, img2):

    # Read the two images
    I1 = cv2.imread(img1)
    I2 = cv2.imread(img2)

    # Resizing image if both are in different shapes
    I2 = cv2.resize(I2, (I1.shape[1], I1.shape[0]))

    # Seperating channels
    iR1 = I1.copy()
    iR1[:, :, 1] = iR1[:, :, 2] = 0
    iR2 = I2.copy()
    iR2[:, :, 1] = iR2[:, :, 2] = 0

    iG1 = I1.copy()
    iG1[:, :, 0] = iG1[:, :, 2] = 0
    iG2 = I2.copy()
    iG2[:, :, 0] = iG2[:, :, 2] = 0

    iB1 = I1.copy()
    iB1[:, :, 0] = iB1[:, :, 1] = 0
    iB2 = I2.copy()
    iB2[:, :, 0] = iB2[:, :, 1] = 0

    shape = (I1.shape[0], I1.shape[1])

    # Wavelet transformation on channel
    outImageR = channelTransform(iR1, iR2, shape)
    outImageG = channelTransform(iG1, iG2, shape)
    outImageB = channelTransform(iB1, iB2, shape)

    #Perform average fusion
    outImage = (outImageR + outImageG + outImageB) / 3

    #Normalize the image
    outImage = np.multiply(np.divide(outImage - np.min(outImage), (np.max(outImage) - np.min(outImage))), 255)
    outImage = outImage.astype(np.uint8)

    return outImage

In [ ]:
# Define the folder path
base_folder = '/content/drive/MyDrive/dataset/'
# Loop from 1 to 512
for i in range(1, 513):
    # Construct the file paths for visual and NIR images
    visual_path = f'/content/drive/MyDrive/dataset/rgb/image_{i}.jpg'
    nir_path = f'/content/drive/MyDrive/dataset/nir/nir_{i}.jpg'

    # Load the visual and NIR images
    visual_image = cv2.imread(visual_path)
    nir_image = cv2.imread(nir_path)

    #perform fusion of visual and infrared image
    fused_image = fusion(visual_path, nir_path)

    # Save the fused image
    fused_output_path = f'/content/drive/MyDrive/dwt_fn/dwt_fused_{i}.jpg'
    cv2.imwrite(fused_output_path, fused_image)

print("Image fusion complete.")

Image fusion complete.


In [ ]:
import cv2
import numpy as np
import pywt
import math

# Define the folder path
base_folder = '/content/drive/MyDrive/dataset/'

# Loop from 1 to 512
for i in range(1, 513):
    # Construct the file paths for visual and NIR images
    visual_path = f'/content/drive/MyDrive/dataset/rgb/image_{i}.jpg'
    nir_path = f'/content/drive/MyDrive/dataset/nir/nir_{i}.jpg'
    # Construct the file paths for fused image
    fused_img_path = f'/content/drive/MyDrive/dwt_fn/dwt_fused_{i}.jpg'
    # Load the visual and NIR images
    visual_image = cv2.imread(visual_path)
    nir_image = cv2.imread(nir_path)
    # Load the fused image and check if it was loaded successfully
    fused_image = cv2.imread(fused_img_path)

    # Preprocessing - Resize the images to a common size
    new_size = (visual_image.shape[1], visual_image.shape[0])
    visual_image = cv2.resize(visual_image, new_size)
    nir_image = cv2.resize(nir_image, new_size)

    # Resize fused image to match the dimensions of visual image
    fused_image = cv2.resize(fused_image, new_size)

    # Calculate entropy for each channel
    channel_visual_entropy = []
    channel_nir_entropy = []
    channel_fused_entropy = []

    for channel in range(visual_image.shape[2]):
        channel_visual_entropy.append(-np.sum(visual_image[:, :, channel] * np.log2(visual_image[:, :, channel] + np.finfo(float).eps)))
        channel_nir_entropy.append(-np.sum(nir_image[:, :, channel] * np.log2(nir_image[:, :, channel] + np.finfo(float).eps)))
        channel_fused_entropy.append(-np.sum(fused_image[:, :, channel] * np.log2(fused_image[:, :, channel] + np.finfo(float).eps)))

    visual_entropy.append(np.abs(channel_visual_entropy))
    nir_entropy.append(np.abs(channel_nir_entropy))
    fused_entropy.append(np.abs(channel_fused_entropy))

    # Calculate MSE for each channel
    visual_mse = []
    mse_values.append([])
    for channel in range(visual_image.shape[2]):
        mse = np.mean((visual_image[:, :, channel] - fused_image[:, :, channel]) ** 2)
        visual_mse.append(mse)
        mse_values[i-1].append(mse)

    # Calculate PSNR for each channel
    psnr = []
    for channel in range(visual_image.shape[2]):
        max_pixel_value = 255  # Assuming 8-bit images
        psnr_value = 10 * np.log10((max_pixel_value ** 2) / visual_mse[channel])
        psnr.append(psnr_value)

    psnr_values.append(psnr)

# Print or use the metric values as needed
for i in range(512):
    print(f'Image {i + 1}:')
    print(f'Visual Entropy: {visual_entropy[i]}')
    print(f'NIR Entropy: {nir_entropy[i]}')
    print(f'Fused Entropy: {fused_entropy[i]}')
    print(f'MSE: {mse_values[i]}')
    print(f'PSNR: {psnr_values[i]}')
    print('-------------------------------------')


Image 1:
Visual Entropy: [7.79258189e+07 9.93604955e+07 1.02693709e+08]
NIR Entropy: [1.37448720e+07 2.32577479e+08 2.58838207e+08]
Fused Entropy: [1.20320824e+08 1.42065926e+08 2.58494976e+08]
MSE: [74.81660461425781, 71.81655375162761, 79.22938028971355]
PSNR: [29.39082366050791, 29.56855799990992, 29.14194101859288]
-------------------------------------
Image 2:
Visual Entropy: [1.26762137e+08 1.43524687e+08 1.45488338e+08]
NIR Entropy: [1.32104835e+08 2.82316411e+08 2.03667957e+08]
Fused Entropy: [2.39728838e+08 2.33670989e+08 2.46060107e+08]
MSE: [60.7803700764974, 59.10632832845052, 91.76523844401042]
PSNR: [30.29317020805273, 30.414463789546502, 28.504021634464628]
-------------------------------------
Image 3:
Visual Entropy: [1.51109187e+08 1.57356201e+08 1.54817157e+08]
NIR Entropy: [8.15048425e+07 2.53165082e+08 2.75747172e+08]
Fused Entropy: [2.15869940e+08 2.11864571e+08 3.01248832e+08]
MSE: [68.53782653808594, 67.31111145019531, 97.34497578938802]
PSNR: [29.77150032841213

In [ ]:
for i in range(1, 513):
    # Calculate the average entropy for the three channels
    average_visual_entropy = np.mean(visual_entropy[i])
    average_nir_entropy = np.mean(nir_entropy[i])
    average_fused_entropy = np.mean(fused_entropy[i])
    # Calculate the absolute value of entropy for each channel
    average_visual_entropy = np.abs(average_visual_entropy)
    average_nir_entropy = np.abs(average_nir_entropy)
    average_fused_entropy = np.abs(average_fused_entropy)

    # Calculate the average MSE for the three channels
    average_mse = np.mean(mse_values[i-1])

    # Calculate the average PSNR for the three channels
    average_psnr = np.mean(psnr_values[i-1])

    # Print or use the average metric values
    print(f'Image {i }:')
    print(f'Average Visual Entropy: {average_visual_entropy:.2f}')
    print(f'Average NIR Entropy: {average_nir_entropy:.2f}')
    print(f'Average Fused Entropy: {average_fused_entropy:.2f}')
    print(f'Average MSE: {average_mse:.2f}')
    print(f'Average PSNR: {average_psnr:.2f}')
    print('-------------------------------------')

Image 1:
Average Visual Entropy: 138591720.48
Average NIR Entropy: 206029734.24
Average Fused Entropy: 239819978.11
Average MSE: 75.29
Average PSNR: 29.37
-------------------------------------
Image 2:
Average Visual Entropy: 154427515.02
Average NIR Entropy: 203472365.73
Average Fused Entropy: 242994447.71
Average MSE: 70.55
Average PSNR: 29.74
-------------------------------------
Image 3:
Average Visual Entropy: 112473472.47
Average NIR Entropy: 198735457.65
Average Fused Entropy: 224643598.11
Average MSE: 77.73
Average PSNR: 29.29
-------------------------------------
Image 4:
Average Visual Entropy: 185297217.60
Average NIR Entropy: 200508243.40
Average Fused Entropy: 249585793.47
Average MSE: 58.53
Average PSNR: 30.66
-------------------------------------
Image 5:
Average Visual Entropy: 159390036.32
Average NIR Entropy: 194461236.60
Average Fused Entropy: 241893009.03
Average MSE: 61.31
Average PSNR: 30.38
-------------------------------------
Image 6:
Average Visual Entropy: 13

#CV2

In [ ]:
import cv2
import numpy as np
import pywt

# Define the folder path
base_folder = '/content/drive/MyDrive/dataset/'

# Loop from 1 to 512
for i in range(1, 513):
    # Construct the file paths for visual and NIR images
    visual_path = f'/content/drive/MyDrive/dataset/rgb/image_{i}.jpg'
    nir_path = f'/content/drive/MyDrive/dataset/nir/nir_{i}.jpg'

    # Load the visual and NIR images
    visual_image = cv2.imread(visual_path)
    nir_image = cv2.imread(nir_path)

    # Preprocessing - Resize the images to a common size
    new_size = (visual_image.shape[1], visual_image.shape[0])
    visual_image = cv2.resize( visual_image, new_size)
    nir_image = cv2.resize(nir_image, new_size)

    # Perform image fusion by taking a weighted average
    alpha = 1.0
    beta = 0.5
    fused_image = cv2.addWeighted(visual_image, alpha, nir_image, beta, 0)

    # Save the fused image
    fused_output_path = f'/content/drive/MyDrive/cv_fusion/drt_fusn_{i}.jpg'
    cv2.imwrite(fused_output_path, fused_image)

print("Image fusion complete.")

Image fusion complete.


In [ ]:
import cv2
import numpy as np
import pywt
import math

# Define the folder path
base_folder = '/content/drive/MyDrive/dataset/'

# Loop from 1 to 512
for i in range(1, 513):
    # Construct the file paths for visual and NIR images
    visual_path = f'/content/drive/MyDrive/dataset/rgb/image_{i}.jpg'
    nir_path = f'/content/drive/MyDrive/dataset/nir/nir_{i}.jpg'
    # Construct the file paths for fused image
    fused_img_path = f'/content/drive/MyDrive/cv_fusion/drt_fusn_{i}.jpg'

    # Load the visual and NIR images
    visual_image = cv2.imread(visual_path)
    nir_image = cv2.imread(nir_path)
    # Load the fused image and check if it was loaded successfully
    fused_image = cv2.imread(fused_img_path)

    # Preprocessing - Resize the images to a common size
    new_size = (visual_image.shape[1], visual_image.shape[0])
    visual_image = cv2.resize(visual_image, new_size)
    nir_image = cv2.resize(nir_image, new_size)
    # Initialize lists for entropy values

    # Resize fused image to match the dimensions of visual image
    fused_image = cv2.resize(fused_image, new_size)

    # Calculate entropy for each channel
    channel_visual_entropy = []
    channel_nir_entropy = []
    channel_fused_entropy = []

    for channel in range(visual_image.shape[2]):
        channel_visual_entropy.append(-np.sum(visual_image[:, :, channel] * np.log2(visual_image[:, :, channel] + np.finfo(float).eps)))
        channel_nir_entropy.append(-np.sum(nir_image[:, :, channel] * np.log2(nir_image[:, :, channel] + np.finfo(float).eps)))
        channel_fused_entropy.append(-np.sum(fused_image[:, :, channel] * np.log2(fused_image[:, :, channel] + np.finfo(float).eps)))

    visual_entropy.append(np.abs(channel_visual_entropy))
    nir_entropy.append(np.abs(channel_nir_entropy))
    fused_entropy.append(np.abs(channel_fused_entropy))

    # Calculate MSE for each channel
    visual_mse = []
    mse_values.append([])
    for channel in range(visual_image.shape[2]):
        mse = np.mean((visual_image[:, :, channel] - fused_image[:, :, channel]) ** 2)
        visual_mse.append(mse)
        mse_values[i-1].append(mse)

    # Calculate PSNR for each channel
    psnr = []
    for channel in range(visual_image.shape[2]):
        max_pixel_value = 255  # Assuming 8-bit images
        psnr_value = 10 * np.log10((max_pixel_value ** 2) / visual_mse[channel])
        psnr.append(psnr_value)

    psnr_values.append(psnr)

# Print or use the metric values as needed
for i in range(512):
    print(f'Image {i + 1}:')
    print(f'Visual Entropy: {visual_entropy[i]}')
    print(f'NIR Entropy: {nir_entropy[i]}')
    print(f'Fused Entropy: {fused_entropy[i]}')
    print(f'MSE: {mse_values[i]}')
    print(f'PSNR: {psnr_values[i]}')
    print('-------------------------------------')


Image 1:
Visual Entropy: [7.79258189e+07 9.93604955e+07 1.02693709e+08]
NIR Entropy: [1.37448720e+07 2.32577479e+08 2.58838207e+08]
Fused Entropy: [1.20320824e+08 1.42065926e+08 2.58494976e+08]
MSE: [73.09752400716145, 111.88347880045573, 85.36891682942708]
PSNR: [29.491776942826696, 27.64314399411241, 28.817805897892978]
-------------------------------------
Image 2:
Visual Entropy: [1.26762137e+08 1.43524687e+08 1.45488338e+08]
NIR Entropy: [1.32104835e+08 2.82316411e+08 2.03667957e+08]
Fused Entropy: [2.39728838e+08 2.33670989e+08 2.46060107e+08]
MSE: [62.43644714355469, 116.33998107910156, 85.76992797851562]
PSNR: [30.176421786788286, 27.473513720368356, 28.79745315469592]
-------------------------------------
Image 3:
Visual Entropy: [1.51109187e+08 1.57356201e+08 1.54817157e+08]
NIR Entropy: [8.15048425e+07 2.53165082e+08 2.75747172e+08]
Fused Entropy: [2.15869940e+08 2.11864571e+08 3.01248832e+08]
MSE: [55.48252868652344, 114.59657287597656, 68.45476786295573]
PSNR: [30.68924114

In [ ]:
for i in range(1, 513):
    # Calculate the average entropy for the three channels
    average_visual_entropy = np.mean(visual_entropy[i])
    average_nir_entropy = np.mean(nir_entropy[i])
    average_fused_entropy = np.mean(fused_entropy[i])

    # Calculate the average MSE for the three channels
    average_mse = np.mean(mse_values[i-1])

    # Calculate the average PSNR for the three channels
    average_psnr = np.mean(psnr_values[i-1])

    # Print or use the average metric values
    print(f'Image {i }:')
    print(f'Average Visual Entropy: {average_visual_entropy:.2f}')
    print(f'Average NIR Entropy: {average_nir_entropy:.2f}')
    print(f'Average Fused Entropy: {average_fused_entropy:.2f}')
    print(f'Average MSE: {average_mse:.2f}')
    print(f'Average PSNR: {average_psnr:.2f}')
    print('-------------------------------------')

Image 1:
Average Visual Entropy: 138591720.48
Average NIR Entropy: 206029734.24
Average Fused Entropy: 239819978.11
Average MSE: 90.12
Average PSNR: 28.65
-------------------------------------
Image 2:
Average Visual Entropy: 154427515.02
Average NIR Entropy: 203472365.73
Average Fused Entropy: 242994447.71
Average MSE: 88.18
Average PSNR: 28.82
-------------------------------------
Image 3:
Average Visual Entropy: 112473472.47
Average NIR Entropy: 198735457.65
Average Fused Entropy: 224643598.11
Average MSE: 79.51
Average PSNR: 29.34
-------------------------------------
Image 4:
Average Visual Entropy: 185297217.60
Average NIR Entropy: 200508243.40
Average Fused Entropy: 249585793.47
Average MSE: 93.66
Average PSNR: 28.55
-------------------------------------
Image 5:
Average Visual Entropy: 159390036.32
Average NIR Entropy: 194461236.60
Average Fused Entropy: 241893009.03
Average MSE: 73.20
Average PSNR: 29.80
-------------------------------------
Image 6:
Average Visual Entropy: 13

#Unet

In [ ]:
!pip install git+https://github.com/karolzak/keras-unet
import numpy as np
import cv2
from keras_unet.models import custom_unet
from keras_unet.utils import plot_imgs
from PIL import Image
import matplotlib.pyplot as plt

  Cloning https://github.com/karolzak/keras-unet to /tmp/pip-req-build-_bge6ctf
  Running command git clone --filter=blob:none --quiet https://github.com/karolzak/keras-unet /tmp/pip-req-build-_bge6ctf
  Resolved https://github.com/karolzak/keras-unet to commit 9b7aff5247fff75dc4e2a11ba9c45929b9166d1f
  Preparing metadata (setup.py) ... done
  Created wheel for keras-unet: filename=keras_unet-0.1.2-py3-none-any.whl size=16992 sha256=f43d68d31da72e1a197b7d9112fc062c5ae075757c3a5dc0516541a8a9997f1f
  Stored in directory: /tmp/pip-ephem-wheel-cache-fsz5lc7t/wheels/75/74/50/939bc5a87c19aad0250aab7fc237e7f8157bfe3ceafde19a3c
Successfully built keras-unet
-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [ ]:
import cv2
import numpy as np
import pywt

# Define the folder path
base_folder = '/content/drive/MyDrive/dataset/'

# Create a U-Net model for segmentation
model = custom_unet(
    input_shape=(512, 512, 6),
    use_batch_norm=False,
    num_classes=1,  # For binary segmentation
    filters=32,
    dropout=0.2,
    output_activation='sigmoid'
)

# Loop from 1 to 512
for i in range(1, 513):
    # Construct the file paths for visual and NIR images
    visual_path = f'/content/drive/MyDrive/dataset/rgb/image_{i}.jpg'
    nir_path = f'/content/drive/MyDrive/dataset/nir/nir_{i}.jpg'

    # Load the visual and NIR images
    visual_image = cv2.imread(visual_path)
    nir_image = cv2.imread(nir_path)

    # Normalize the images
    visual_image = visual_image / 255.0
    nir_image = nir_image / 255.0

    # Resize the images to match the model's input shape (512x512 pixels)
    desired_shape = (512, 512)
    visual_image = cv2.resize(visual_image, desired_shape)
    nir_image = cv2.resize(nir_image, desired_shape)

    # Concatenate visual and NIR images
    concatenated_input = np.concatenate([visual_image, nir_image], axis=-1)

    # Predict segmentation masks for the concatenated input
    segmentation_masks = model.predict(np.array([concatenated_input]))

    # Apply segmentation masks to images
    segmented_visual_image = visual_image * segmentation_masks
    segmented_nir_image = nir_image * segmentation_masks

    # Combine segmented images to obtain the fused result
    fused_image = 1.0 * segmented_visual_image + 0.4 * segmented_nir_image

    # Post-process the fused image
    fused_image = (fused_image * 255).astype(np.uint8)
    fused_image = fused_image[0]

    # Save or display the fused image
    fused_output_path = f'/content/drive/MyDrive/unet_fusion1/unet_fusn_{i}.jpg'
    cv2.imwrite(fused_output_path, fused_image)

print("Image fusion complete.")

1/1 [==============================] - 0s 29ms/step
Image fusion complete.


In [ ]:
import cv2
import numpy as np
import math

# Define the folder path
base_folder = '/content/drive/MyDrive/dataset/'

# Lists to store the metric values for each pair of images
entropy_values = []
mse_values = []
psnr_values = []
visual_entropy = []
nir_entropy = []
fused_entropy = []
# Loop from 1 to 512
for i in range(1, 513):
    # Construct the file paths for visual and NIR images
    visual_path = f'/content/drive/MyDrive/dataset/rgb/image_{i}.jpg'
    nir_path = f'/content/drive/MyDrive/dataset/nir/nir_{i}.jpg'
    # Construct the file paths for fused image
    fused_img_path = f'/content/drive/MyDrive/unet_fusion1/unet_fusn_{i}.jpg'

    # Load the visual and NIR images
    visual_image = cv2.imread(visual_path)
    nir_image = cv2.imread(nir_path)
    # Load the fused image and check if it was loaded successfully
    fused_image = cv2.imread(fused_img_path)

    # Preprocessing - Resize the images to a common size
    new_size = (visual_image.shape[1], visual_image.shape[0])
    visual_image = cv2.resize(visual_image, new_size)
    nir_image = cv2.resize(nir_image, new_size)
    # Initialize lists for entropy values

    # Resize fused image to match the dimensions of visual image
    fused_image = cv2.resize(fused_image, new_size)

    # Calculate entropy for each channel
    channel_visual_entropy = []
    channel_nir_entropy = []
    channel_fused_entropy = []

    for channel in range(visual_image.shape[2]):
        channel_visual_entropy.append(-np.sum(visual_image[:, :, channel] * np.log2(visual_image[:, :, channel] + np.finfo(float).eps)))
        channel_nir_entropy.append(-np.sum(nir_image[:, :, channel] * np.log2(nir_image[:, :, channel] + np.finfo(float).eps)))
        channel_fused_entropy.append(-np.sum(fused_image[:, :, channel] * np.log2(fused_image[:, :, channel] + np.finfo(float).eps)))

    visual_entropy.append(np.abs(channel_visual_entropy))
    nir_entropy.append(np.abs(channel_nir_entropy))
    fused_entropy.append(np.abs(channel_fused_entropy))

    # Calculate MSE for each channel
    visual_mse = []
    mse_values.append([])
    for channel in range(visual_image.shape[2]):
        mse = np.mean((visual_image[:, :, channel] - fused_image[:, :, channel]) ** 2)
        visual_mse.append(mse)
        mse_values[i-1].append(mse)

    # Calculate PSNR for each channel
    psnr = []
    for channel in range(visual_image.shape[2]):
        max_pixel_value = 255  # Assuming 8-bit images
        psnr_value = 10 * np.log10((max_pixel_value ** 2) / visual_mse[channel])
        psnr.append(psnr_value)

    psnr_values.append(psnr)

# Print or use the metric values as needed
for i in range(512):
    print(f'Image {i + 1}:')
    print(f'Visual Entropy: {visual_entropy[i]}')
    print(f'NIR Entropy: {nir_entropy[i]}')
    print(f'Fused Entropy: {fused_entropy[i]}')
    print(f'MSE: {mse_values[i]}')
    print(f'PSNR: {psnr_values[i]}')
    print('-------------------------------------')


Image 1:
Visual Entropy: [1.26762137e+08 1.43524687e+08 1.45488338e+08]
NIR Entropy: [1.32104835e+08 2.82316411e+08 2.03667957e+08]
Fused Entropy: [9.49519358e+07 1.42469341e+08 1.28112132e+08]
MSE: [110.24582926432292, 102.11837768554688, 87.92812093098958]
PSNR: [27.70718192313086, 28.039764541463068, 28.689525687063117]
-------------------------------------
Image 2:
Visual Entropy: [1.51109187e+08 1.57356201e+08 1.54817157e+08]
NIR Entropy: [8.15048425e+07 2.53165082e+08 2.75747172e+08]
Fused Entropy: [9.89193936e+07 1.45674379e+08 1.50241656e+08]
MSE: [110.38123575846355, 100.88085428873698, 87.66778055826823]
PSNR: [27.701851090252617, 28.09271609553941, 28.702403489924606]
-------------------------------------
Image 3:
Visual Entropy: [1.09221895e+08 1.13859876e+08 1.14338646e+08]
NIR Entropy: [8.63234820e+07 2.26464484e+08 2.83418407e+08]
Fused Entropy: [7.43004423e+07 1.11879794e+08 1.27343074e+08]
MSE: [113.06939697265625, 101.83571370442708, 85.37081400553386]
PSNR: [27.59735

In [ ]:
for i in range(512):
    # Calculate the average entropy for the three channels
    average_visual_entropy = np.mean(visual_entropy[i])
    average_nir_entropy = np.mean(nir_entropy[i])
    average_fused_entropy = np.mean(fused_entropy[i])

    # Calculate the average MSE for the three channels
    average_mse = np.mean(mse_values[i])

    # Calculate the average PSNR for the three channels
    average_psnr = np.mean(psnr_values[i])

    # Print or use the average metric values
    print(f'Image {i + 1}:')
    print(f'Average Visual Entropy: {average_visual_entropy:.2f}')
    print(f'Average NIR Entropy: {average_nir_entropy:.2f}')
    print(f'Average Fused Entropy: {average_fused_entropy:.2f}')
    print(f'Average MSE: {average_mse:.2f}')
    print(f'Average PSNR: {average_psnr:.2f}')
    print('-------------------------------------')


Image 1:
Average Visual Entropy: 138591720.48
Average NIR Entropy: 206029734.24
Average Fused Entropy: 121844469.68
Average MSE: 100.10
Average PSNR: 28.15
-------------------------------------
Image 2:
Average Visual Entropy: 154427515.02
Average NIR Entropy: 203472365.73
Average Fused Entropy: 131611809.57
Average MSE: 99.64
Average PSNR: 28.17
-------------------------------------
Image 3:
Average Visual Entropy: 112473472.47
Average NIR Entropy: 198735457.65
Average Fused Entropy: 104507770.00
Average MSE: 100.09
Average PSNR: 28.16
-------------------------------------
Image 4:
Average Visual Entropy: 185297217.60
Average NIR Entropy: 200508243.40
Average Fused Entropy: 150236522.33
Average MSE: 99.93
Average PSNR: 28.15
-------------------------------------
Image 5:
Average Visual Entropy: 159390036.32
Average NIR Entropy: 194461236.60
Average Fused Entropy: 132271426.21
Average MSE: 102.42
Average PSNR: 28.04
-------------------------------------
Image 6:
Average Visual Entropy: